In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle
import time
from slugify import slugify
from fuzzywuzzy import fuzz
import urllib
import math

##SET PATH TO SAVE FILES:
savpath="./bands_dicts/"


In [2]:
#a function to have a range of colors for nodes/edges 
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [3]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    #print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [ ]:
artist={}
artist["name"]="steve-aoki"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/tour_miner/"+slugify(artist["name"]+artist["mbid"])+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
print tempL

In [137]:
###HERE WE BUILD A LABEL OWNERSHIP CRUSHER FACTORIZED
with open("../../4_miner/labels_aliases_TOTAL.obj", 'rb') as handle:
          tempRelAll= pickle.load( handle)
def find_owning_label(label):
    
    labrels=[]
    if "label-relation-list" in tempRelAll[label]["label"] :

        for relat in tempRelAll[label]["label"]["label-relation-list"]:
            if "direction" in relat and relat["direction"] == "backward":
                print tempRelAll[label]["label"]["name"],"OWNED BY",relat["target"]," ",tempRelAll[relat["target"]]["label"]["name"]
                labrels.append(relat["target"])
                rum_and_whisky=""
                while rum_and_whisky != "SELF": 
                    rum_and_whisky=find_owning_label(relat["target"])
                print "ENDED LOOP"
                labrels.remove(relat["target"])
                print "LABRELS",labrels
                #if labrels[0]==relat["target"]:
                 #   break
                
            else:
                pass
            
        
        if not labrels :
            print "TOP OF THE TREE FOR ",tempRelAll[label]["label"]["name"]
            return "SELF"
        else:
            return labrels
                


artist={}
artist["name"]="Steve Aoki"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'rb') as handle:
          tempRelz= pickle.load( handle)
#print tempRel["release-list"][0]['label-info-list'][0]['label']['id']
#print tempRel["release-list"][0]['release-event-list'][0]["date"]




for release in tempRelz["release-list"]:
    print"---------------------------"
    print release['label-info-list'][0]['label']['id']
    print release['label-info-list'][0]['label']['name']
    print release['release-event-list'][0]["date"]
    
    #print tempRelAll[release['label-info-list'][0]['label']['id']]["label"]
    try:
        result_lab=find_owning_label(release['label-info-list'][0]['label']['id'])
        print "REZLAB",result_lab 
        print"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    except KeyError as e:
        #print e
        pass

---------------------------
6c690937-c6c3-4c4e-9d3e-c62efa989333
Ultra Records
2012-07-10
Ultra Records OWNED BY f0d2ccb8-a269-491c-bbe3-08bdcd3251b7   Ultra Records, Inc.
TOP OF THE TREE FOR  Ultra Records, Inc.
ENDED LOOP
LABRELS []
TOP OF THE TREE FOR  Ultra Records
REZLAB SELF
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
---------------------------
44510f75-af0c-49b5-ab17-179f38452781
Ministry of Sound (Australia)
2010-05-14
Ministry of Sound (Australia) OWNED BY c48b8ed2-46c5-4e51-9bea-5a37fcd0053b   Ministry of Sound
Ministry of Sound OWNED BY 136021ca-a54d-4451-99c9-6ecf52121b86   Ministry of Sound Recordings Ltd.
Ministry of Sound Recordings Ltd. OWNED BY 9a40374c-c95b-47bc-b353-b2b9728848d3   Sony Music Entertainment UK Limited
Sony Music Entertainment UK Limited OWNED BY b54769e4-b75e-4f60-884a-c4714687bb4c   Sony Music Entertainment
Sony Music Entertainment OWNED BY a57fa9c5-f488-4bae-9eb6-41b734227e90   Sony Corporation of America
TOP OF THE TREE FOR  Sony Corporation of America
ENDED

In [11]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
#title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}
my_nodesdictGLOBAL={}

#LIVENATION CATALOGUE
liste_salles_LN = json.load(open('./JSON_LIVENATION_VENUE.json'))

liste_salle_parsee_LN={}
for salle in liste_salles_LN:
    if liste_salles_LN[salle]["type"]=="venue":
 
            liste_salle_parsee_LN[slugify(liste_salles_LN[salle]["title"])]={"lat":liste_salles_LN[salle]["position"]["lat"],"lng":liste_salles_LN[salle]["position"]["lng"]}
#print liste_salle_parsee_LN


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()

#for artist in db.minedArtists.find({"totalKm": {"$gte": 250000}}) :
for artist in db.minedArtists.find({"name" : "Bad Brains"}) :
 title = artist["name"]   
 if os.path.exists(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        my_nodesdict = {}
        my_nodesdictList=[]
        print "SKIPPED PROCESSING BUT TAKING INTO ACCOUNT ",artist["name"]
        with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
    #print 
        tit=tempL[0].encode("utf-8")
        #edgestop=temp[2]
        my_nodesdictList=tempL[1]
        for node in my_nodesdictList:
            my_nodesdict[node["data"]["id"]]=node

        #print my_nodes
        #pass
 else:
    my_nodesdict = {}
    my_nodes= []
    my_edges = []
    my_nodesdictList=[]
    my_edgesdict = {}
    #colors_for_nodes_and_edges = get_N_HexCol()
    
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    tours = artist.get("tours")    
    singlegigs = artist.get("singleGigs")
    dateini=0
    touring=0
    tour_indice=0
    tour_counter_for_CO2=1
    tour_counter_for_CO2_lock=0
    tmpOLDgig ={}
    with open("../../4_miner/tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
                    tempCO2= pickle.load( handle)
    #print tempCO2
    #print tours
    for tour in tours:
        
        ##WE NEED TO MATCH TOUR INDICES OF CO2 SPECS FILES...
     if tour_counter_for_CO2_lock == 1:
        tour_counter_for_CO2+=1
     tour_counter_for_CO2_lock=0
     tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
     #print tour_color
     #print tour
     gig_indice =0
     dates = tour.get("gigs")
     for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            
            node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    }
        else:
                
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : math.sqrt(float(my_nodesdict[idd]["data"].get("weight"))+1) ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight"))+1)**2)+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                    #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    
                    }
        if dateini == 0:
                dateini =1
        else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                
                
                
                notesE=""
                notesE+="distance : "
                try: 
                    notesE+=str(dates[gig_indice]["distanceToNextGig"])
                except KeyError:
                    notesE+="dates[gig_indice][distanceToNextGig] NOT AVAILABLE"
                notesE+=" km  \n"
                notesE+="source : "
                notesE+=str(my_nodes[-1]["data"]["id"])
                notesE+="  target : "
                notesE+=str(node["id"])
                notesE+="  \ndatesource : "
                notesE+=dates[gig_indice -1]["datetime"].isoformat()
                notesE+="  \ndatetarget : "
                notesE+=dates[gig_indice]["datetime"].isoformat()
                notesE+="  \ngroup: "
                notesE+=str(int(tour_indice)+1)
                #notesE+=
                
                
                #print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    "notes" : notesE,
                    #"weight" : float(e["weight"]),
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    "group" : str(tour_counter_for_CO2)
                    
                }
                #print edge
                #ADDING EDGES GROUPS/CO2 HERE
                
                if tour["nbGigs"] > 2:
                    try:
                        if tempCO2[tour_counter_for_CO2]['tourdist'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']:
                            
                            edge["group"]+=" DASHED2"
                            edge["notes"]+="  \n tournée grandement optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 2"
                        elif (tempCO2[tour_counter_for_CO2]['tourdist'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']) and (tempCO2[tour_counter_for_CO2]['tourdist'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdistOpt']):
                            
                            edge["group"]+=" DASHED1"
                            edge["notes"]+="  \n  tournée optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 1"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-2"
                            edge["notes"]+="  \n tournée déjà grandement optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -2"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdist'] and tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-1"
                            edge["notes"]+="  \n  tournée déjà optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -1"
                        
                        else:

                            edge["notes"]+="  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                        tour_counter_for_CO2_lock=1
                    except KeyError as e:
                        print e
                else:
                    #print "TOUR NON COMPTE ", tour_indice, "//",tour_counter_for_CO2
                    continue
                    
                    
                    
                my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
     tour_indice +=1
        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    
    #ADDING LIVENATION DETECTOR

    for node in my_nodesdictList:
        for salle in liste_salle_parsee_LN:
            if fuzz.partial_ratio( node['data']['name'], salle)>=80:
                #print "ONE HEERE"
                if round(node['data']['lat'],2)==round(liste_salle_parsee_LN[salle]["lat"],2) and round(node['data']['lng'],2)==round(liste_salle_parsee_LN[salle]["lng"],2) :
                    #print "CONFIRMED LIVENATION!"
                    node['data']['color']="#ff0000"
                    node['data']["notes"]+="  \nLivenation venue"
        #if liste_salle_parsee_LN
        
        
    #ADDING EDGES WEIGHT
    terp= map(lambda x : str(sorted([x["data"]["source"],x["data"]["target"]])),my_edges)
    
    #print terp
    from collections import Counter
    c= Counter()
    for ter in terp:
        c[ter]+=1
    #print c
    for edge in my_edges:
        if str(sorted([edge["data"]["source"],edge["data"]["target"]])) not in terp:
            print "BUG!!!!"
        else:
            if edge["data"]["source"] == edge["data"]["target"]:
                #print c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))]
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(1)
                #print "a"
            else:
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])
                #print "b"
    #print my_edges

    
    
    
    

    titredugraphe =artist["name"]+"/BETA_0.4"+"  \nDistance totale parcourue par l'artiste: "+str(tempCO2["totalArtistTourDist"])+" km"
    titredugraphe += "  \nDistance calculee par Concorde: "+str(tempCO2["totalArtistOptTourDist"])+" km"
    if  "OptArtistPercent" in tempCO2 :
         rateOpt=0
         rateOpt=float(tempCO2["OptArtistPercent"])
         titredugraphe+="  \nPourcentage d'optimisation global calcule:"+str(rateOpt)+" %"
         if rateOpt > 10: 
             titredugraphe+="  \nMARGE D'OPTIMISATION IMPORTANTE"
         elif rateOpt < -10:
              titredugraphe+="  \nTOURNEE DEJA OPTIMISEE"
         else: 
             titredugraphe+="  \nGLOBALEMENT IDENTIQUE"
         
         
    with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump([titredugraphe,my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    
            
    ###CREATE NODE GLOBAL DICT


SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  Bad Brains


In [ ]:
     with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)

In [31]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print take(5,my_nodesdictGLOBAL.iteritems())

[('pete-s-place32-117', {'data': {'count': 1, 'end': '2017-04-01T19:00:00', 'name': 'pete-s-place', 'weight': 1.0, 'color': '#7f523f', 'notes': '  \n List of the bands that played in this venue:  \nblack-joe-lewis  \n TOTAL: 1 Bands', 'start': '2017-03-31T19:00:00', 'lat': 32.7677778, 'lng': -117.0222222, 'id': 'pete-s-place32-117'}}), ('tucson-music-hall32-110', {'data': {'count': 35, 'end': '2018-01-15T14:00:00', 'name': 'tucson-music-hall', 'weight': 7.158396516477644, 'color': '#3f7f6c', 'notes': '  \n List of the bands that played in this venue:  \ncherish-the-ladies  \nitzhak-perlman  \nbill-maher  \ncurtis-stigers  \njonathan-richman  \nthe-lettermen  \npiotr-ilich-chaikovskii  \nthe-midtown-men  \njoseph-haydn  \ngeorge-lopez  \nsuper-diamond  \njane-s-addiction  \nlindsey-stirling  \narturo-sandoval  \nlewis-black  \ndon-giovanni  \njerry-seinfeld  \nkt-tunstall  \nvideo-games-live  \nthe-ten-tenors  \nbellamy-brothers-band  \nbrian-regan  \nmichael-feinstein  \nrain  \njackso

In [7]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [8]:
print len(my_nodesdictGLOBAL)

0


In [9]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'd5zS9GSMaPZsa32KWW4lrvvz87AY9rBtZg4ENf3uKQ3', u'userId': u'BgWEdXXGnvEq2Hodg'}}


In [10]:
###IMPORT ALL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

file= "./bands_dicts/dead-kennedys37c78aeb-d196-42b5-b991-6afb4fc9bc2e.obj"
with open(file, 'rb') as handle:
          temp= pickle.load( handle)
    #print 
tit=temp[0].encode("utf-8")
edgestop=temp[2]
nodestop=temp[1]
    #print edgestop
    #for edge in edgestop:
    #    assert edge["data
while True:
        try:
            #print
            create_topogram(tit, nodestop, edgestop)
        except TypeError:
            continue
        break
print 'DONE'
        

/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
Creating topogram 'Dead Kennedys/BETA_0.4  
Distance totale parcourue par l'artiste: 49960.3182228 km  
Distance calculee par Concorde: 50365.355494 km  
Pourcentage d'optimisation global calcule:-0.810717957032 %  
GLOBALEMENT IDENTIQUE'
{u'status': u'success', 'status_code': 200, u'data': u'HsyMd7vY7CqkHJLqZ', u'statusCode': 201}
Creating topogram 'Dead Kennedys/BETA_0.4  
Distance totale parcourue par l'artiste: 49960.3182228 km  
Distance calculee par Concorde: 50365.355494 km  
Pourcentage d'optimisation global calcule:-0.810717957032 %  
GLOBALEMENT IDENTIQUE'


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Dead Kennedys/BETA_0.4  \nDistance totale parcourue par l'artiste: 49960.3182228 km  \nDistance calculee par Concorde: 50365.355494 km  \nPourcentage d'optimisation global calcule:-0.810717957032 %  \nGLOBALEMENT IDENTIQUE", u'userId': u'BgWEdXXGnvEq2Hodg', u'_id': u'HsyMd7vY7CqkHJLqZ', u'slug': u'dead-kennedysbeta_04-distance-totale-parcourue-par-lartiste-499603182228-km-distance-calculee-par-concorde-50365355494-km-pourcentage-doptimisation-global-calcule-0810717957032-globalement-identique', u'createdAt': u'2018-06-02T09:01:03.660Z'}, u'statusCode': 200}
topogram ID : HsyMd7vY7CqkHJLqZ
96 nodes created.
106 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/HsyMd7vY7CqkHJLqZ
DONE


In [50]:
###IMPORT THE GLOBAL VENUE  GRAPH
my_nodesdictList=[]
nodestopo=[]
edgestopo=[]
with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictGLOBAL[key]["data"].pop('count')
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictList.append(value)
        
nodestopo=sorted(my_nodesdictList, key= lambda (v) : v['data']['weight'], reverse=True)[:5000]
print len(nodestopo)



title="global venue graph /BETA 0.3"

while True:
        try:
            print nodestopo
            create_topogram(title, nodestopo, edgestopo)
        
        except TypeError:
            continue
        break
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


5000 nodes deleted
5000 nodes created.


ValueError: No JSON object could be decoded